In [1]:
from tqdm.notebook import tqdm

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("abacus")

SLG [I]: Checking config correctness... Config correct!
SLG [I]: Dataset loaded from disk (dataset file)
SLG [I]: Vocabulary loaded from disk


All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/abacus/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


SLG [I]: Paradigmatizer loaded from disk


In [3]:
meno_corpus = semiotic.corpus.train.filter(lambda paragraph: "meno" in paragraph["text"])

In [4]:
meno_paradigms = []
for passage in tqdm(meno_corpus):
    text = semiotic(passage["text"], paradigms=False)
    for i,token in enumerate(text.chain.tokens):
        if token.label == "meno":
            meno_parad = semiotic.paradigmatic.paradigmatizer(text.chain, i)
            meno_paradigms.append(meno_parad)

  0%|          | 0/1960 [00:00<?, ?it/s]

In [10]:
sent = meno_corpus[2]["text"]
sent_sq = semiotic(sent)

In [22]:
bla = semiotic.paradigmatic.paradigmatizer(sent_sq.chain, 0)

In [23]:
bla

['.', '1', '4', '«', '10', '-', '2', '3', '*', '|', 'e', '•', '16', '5', '7', '20', '25', '11', '9', '□', 'censo', 'la', '!', 'di', '[', "'", '^', '..', 'le', '12', 'X', '?', '»', 'che', '14', '100', '/', 'j', 'meno', ').', ',', '>', '<', '//', '—', 'a', '6', ';', '15', '■', '13', '40', '80', '41', '21', '8', '0', '10000', 'i', 'il', '30', 'dove', '17', '112', 'J', 'più', '50', '£', '’', 'fanno', ']', '19', ')', '34', 'radice', 'c', '(', 'r', '24', '26', 'sic', ':', '90', '120', 'chose', 'I', '161', '33', 'Adunque', '841', '58', '].', 'E', '39', 'è', 'se', '68', '53', '35', '18', '’.', 'una', '250', '625', 'moltiplicha', '47', 'noj', '48', 'T', 'chosa', '31', '84', '110', '।', 'per', '49', 'Ora', 'prima', '56', '70', 'x', 'viè', '29', '101', '172', '72', 'da', 'quale', '37', 'dipoj', 'v', '74', '360', '81', '69', '256', '28', '64', 'cen', '196', 'cioè', '144', 'gli', 'chosj', '98', 'censj', 'numero', 'essere', '22', '54', '169', '/.', '.,', 'questo', 'in', '60', 'dj', '103', '86', 'fia

In [26]:
bla2 = sent_sq.chain.tokens[0].paradigm

In [48]:
bla.entropy == bla2.productivity

True

In [51]:
bla.entropy

5.345339

In [46]:
[i-j for i,j in zip(bla.probs,bla2.probs) if i!=j]

[]

In [12]:
sent_sq.parad_chain.df()

,|_0,Adunque_1,10_2,meno_3,uno_4,censo_5,è_6,igualj_7,a_8,16_9,e_10,1_11,censo_12,meno_13
0,.,e,16,meno,uno,censo,è,igualj,a,10,meno,uno,censo,e
1,1,che,10,e,1,chubo,sono,iguale,al,censo,chose,1,chosa,di
2,4,meno,100,che,il,cienso,sia,iguali,allo,100,e,16,chubo,.
3,«,a,25,chose,un,chosa,e,da,ad,1,censo,10,chose,meno
4,10,moltiplicha,1,a,di,cubo,",",equale,in,14,chosa,4,cosa,","
5,-,da,20,per,lo,cen,sarà,eguale,alla,20,censj,di,cienso,dj
6,2,fanno,4,censo,una,igualj,saranno,insino,alle,5,censi,11,1,1
7,3,per,11,più,ognj,che,fia,dando,di,25,più,3,10,10
8,*,ognj,5,è,10,quadrato,censo,10,da,11,1,25,censj,;
9,|,araj,12,",",16,censj,sonno,che,",",1000,parte,2,denaio,16


In [5]:
sent = "la sua radici, vai la cosa, meno il dimeggamento de le cose, che fu 3."

In [6]:
sent_sq = semiotic(sent)

In [7]:
sent_sq.parad_chain.df()

,la_0,sua_1,radici_2,",_3",vai_4,la_5,cosa_6,",_7",meno_8,il_9,dimeggamento_10,de_11,le_12,cose_13,",_14",che_15,fu_16,3_17,._18
0,la,sua,radice,",",vale,la,cosa,",",vale,il,dimeggamento,de,le,cose,",",che,fu,il,.
1,La,cosa,radici,;,meno,il,chosa,;,meno,del,dimezamento,di,’,radici,;,tanto,sono,2,/
2,il,somma,somma,e,varrà,una,cose,.,più,lo,dimenamento,per,3,chose,più,dunqua,è,5,;
3,Ila,più,cosa,fu,varà,le,somma,vale,fu,el,numero,vale,',cosa,:,adunqua,vale,numero,","
4,Il,vale,metà,de,più,La,radici,de,tranne,collo,dimegamento,delle,la,",",e,e,varà,9,cose
5,della,de,",",è,fu,della,radice,e,e,al,dimezzamento,",",5,parti,quello,cotanto,remane,4,:
6,dela,fu,regola,:,e,alcuna,metà,fu,et,nel,partimento,e,2,co,(,",",vaglano,30,meno
7,fu,",",ragione,vale,et,lo,quantità,è,varà,Ilo,mese,a,Ile,censi,fu,quanto,fa,8,più
8,ala,la,.,cuba,tranne,delle,regola,delle,è,quello,primo,;,4,C,.,et,sirano,6,e
9,mento,delle,chubicha,fa,fa,Ila,sua,fa,cioè,la,censo,fa,alle,partite,".,",chotanto,farà,fu,□


In [18]:
sent = "I have made my plans and I must stick to them"
sent_seq = semiotic(sent)
sent_seq.chain.labels

['I', 'have', 'made', 'my', '[UNK]', 'and', 'I', 'must', 'stick', 'to', 'them']